In [6]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import time

In [7]:
# 크롬 드라이버 가져오기
chrome_driver = ChromeDriverManager().install()
service = Service(chrome_driver)

# 크롤링 옵션 생성
options = webdriver.ChromeOptions()

# 백그라운드 실행 옵션 추가
options.add_argument("headless")

# Velog

## URL

In [8]:
def url_velog(scroll_num=10, period='month'):
    
    """
    Velog(https://velog.io/)에 포함된 'url', 'title'을 가져오는 함수
        Args:
            scroll_num (int): 스크롤 몇 회 내릴지
            period (str): 검색할 기간 | week, month, year
        Returns:
            'url', 'title', 'source' 컬럼을 가진 dataframe
    """
    
    df=pd.DataFrame()
    L_url=[]
    L_title=[]
    L_topic=[]
    L_source=[]
    
    url='https://velog.io/'
    driver = webdriver.Chrome()
    #driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    time.sleep(1)
    
    if period=='week':
        for i in tqdm(range(scroll_num)):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
    
    if period=='month':
        a=driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[3]/div[1]/div[1]/div[2]')
        a.click()
        b=driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[3]/div[1]/div[1]/div[3]/div/div/ul/li[3]')
        b.click()
        time.sleep(3)

        for i in tqdm(range(scroll_num)):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            
    if period=='year':
        a=driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[3]/div[1]/div[1]/div[2]')
        a.click()
        b=driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[3]/div[1]/div[1]/div[3]/div/div/ul/li[4]')
        b.click()
        time.sleep(3)

        for i in tqdm(range(scroll_num)):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(12)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    items = soup.select(".sc-lbhJGD.jhNfXV")
    
    for item in items:
        L_title.append(item.select_one("h4").text)
        L_url.append('https://velog.io'+item.select_one(".sc-jgrJph.dYjA-dM")['href'])
        L_source.append('velog')
    
    if scroll_num!=0:
        
        items = soup.select(".sc-lbhJGD.cBCWUE")

        for item in items:
            L_title.append(item.select_one("h4").text)
            L_url.append('https://velog.io'+item.select_one(".sc-jgrJph.jPHRjo")['href'])
            L_source.append('velog')

        df=pd.concat([df,pd.DataFrame({'url':L_url, 'title':L_title, 'source':L_source})]).reset_index(drop=True)
        driver.close()
    
    return df

In [9]:
df=url_velog(scroll_num=3, period='week')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.03s/it]


In [10]:
df.head(3)

,url,title,source
0,https://velog.io/@lingodingo/토스를-떠나면서,토스를 떠나면서,velog
1,https://velog.io/@turtle601/타입스크립트도-조건문을-쓸-수-있...,타입스크립트도 조건문을 쓸 수 있다??!!? ㄴ(°0°)ㄱ,velog
2,https://velog.io/@beberiche/돈이-없는-취준생은-노트북으로-배...,돈이 없는 취준생은 노트북에 배포를 했다.,velog


## Context

In [11]:
# 본문 스크래핑
def context_tag_velog(url):
    
    """
    Velog(https://velog.io/)에 포함된 url에 해당하는 'context', 'tag'를 가져오는 함수
        Args:
            url : Velog(https://velog.io/)에 포함된 url
        Returns:
            'url', 'context', 'tag' 컬럼을 가진 dataframe
    """
    
    try:
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "lxml") 
        text = soup.find("div", attrs={"class":"atom-one"}).get_text()
        text = text.replace("\n","") #공백 제거
        text = text.replace("\xa0"," ") # 띄어쓰기가 \xa0로 표시돼서 다시 바꿈
        items=soup.select_one("div", attrs={"class":"sc-evcjhq.bIwPM"}).select("a", attrs={"class":"sc-gIDmLj.ftTqiG"})
        tags=[]
        
        for item in items:
            if '/tags' in item['href']:
                tags.append(item.text)
        return text,tags
    
    except:
        return np.nan,np.nan

In [12]:
df['tags']=''
for i in tqdm(range(len(df))):
    df.loc[i,'context'],df.at[i,'tags']=context_tag_velog(df.loc[i,'url'])

100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [00:30<00:00,  2.88it/s]


In [13]:
df.head(3)

,url,title,source,tags,context
0,https://velog.io/@lingodingo/토스를-떠나면서,토스를 떠나면서,velog,[],🚚 떠나면서토스는 나의 두 번째 회사이기도 하면서 참 많은 경험을 겪게 해준 고마운...
1,https://velog.io/@turtle601/타입스크립트도-조건문을-쓸-수-있...,타입스크립트도 조건문을 쓸 수 있다??!!? ㄴ(°0°)ㄱ,velog,[typescript],위 글 은 타입스크립트의 조건부 타입 을 사용해보고 정리하는 글입니다. 📌 조건부 ...
2,https://velog.io/@beberiche/돈이-없는-취준생은-노트북으로-배...,돈이 없는 취준생은 노트북에 배포를 했다.,velog,"[DDNS, ec2, ubuntu, 방화벽, 배포, 포트포워딩]","개요프론트엔드 개발자이지만, 서비스 배포 및 운영에 매우 큰 관심을 가지고 있다. ..."


# Tistory

## URL

In [14]:
def makeUrl_tistory(topic):
    urls=[]
    url=f'https://www.tistory.com/category/{topic}'
    return url

In [15]:
def url_tistory(scroll_num=10,topic=['life','travel','culture','it','sports','current']):
    
    """
    Tistory(https://www.tistory.com/category/{해당 topic})에 포함된 'url', 'title', 'big_topic', 'small_topic'을 가져오는 함수
        Args:
            scroll_num (int): 스크롤 몇 회 내릴지
            topic (list): 크롤링하고 싶은 주제 | ['life','travel','culture','it','sports','current']
        Returns:
            'url', 'title', 'big_topic', 'small_topic', 'source' 컬럼을 가진 dataframe
    """
    
    df=pd.DataFrame()
    
    for n in topic:
        globals()[f'urls_{n}']=makeUrl_tistory(topic=n)
        
    for n in topic:
        L_url=[]
        L_title=[]
        L_topic=[]
        L_label=[]
        
        driver = webdriver.Chrome()
        #driver = webdriver.Chrome(service=service, options=options)
        url=globals()[f'urls_{n}']
        driver.get(url)
        time.sleep(1)

        for i in tqdm(range(scroll_num)):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        items = soup.select(f".\#article\#{n}")

        for item in items:
            L_title.append(item.select_one(".inner_desc_tit").text)
            L_url.append(item['href'])
            L_topic.append(item.select_one(".inner_data").text)
            L_label.append(n)
            
        driver.close()
        print(f'{n}: {len(L_url)}개')
        df=pd.concat([df,pd.DataFrame({'url':L_url,'big_topic':L_label,'small_topic':L_topic, 'title':L_title})]).reset_index(drop=True)
    
    df['source']='tistory'
    return df

In [16]:
df = url_tistory(scroll_num=3,topic=['life','travel','culture','it','sports','current'])

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.02s/it]


life: 36개


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.02s/it]


travel: 48개


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.02s/it]


...

it: 36개


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.02s/it]


sports: 48개


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.02s/it]


current: 48개


In [17]:
df.head(3)

,url,big_topic,small_topic,title,source
0,https://mok0nolg0.tistory.com/2127,life,해외생활,점점 더 스며드는 중이다 (feat.장보기),tistory
1,https://donothurt.tistory.com/28,life,건강,"기침에 좋은 음식,은행 잎 열매 효능 부작용",tistory
2,https://miyah806.tistory.com/entry/93세-엄마의-생신,life,해외생활,엄마의 93세 생신,tistory


## Context

In [18]:
def context_tistory(url):
    
    """
    Tistory(https://www.tistory.com/category/{해당 topic})에 포함된 url에 해당하는 'context'를 가져오는 함수
        Args:
            url : Tistory(https://www.tistory.com/category/{해당 topic})에 포함된 url
        Returns:
            'url', 'context' 컬럼을 가진 dataframe
    """
    
    try:
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "lxml") 
        text = soup.find("div", attrs={"class":"contents_style"}).get_text()
        text = text.replace("\n","") #공백 제거
        text = text.replace("\xa0"," ") # 띄어쓰기가 \xa0로 표시돼서 다시 바꿈
        return text
    
    except:
        return np.nan

In [19]:
for i in tqdm(range(len(df))):
    df.loc[i,'context']=context_tistory(df.loc[i,'url'])

100%|████████████████████████████████████████████████████████████████████████████████| 264/264 [00:40<00:00,  6.53it/s]


In [20]:
df.head(3)

,url,big_topic,small_topic,title,source,context
0,https://mok0nolg0.tistory.com/2127,life,해외생활,점점 더 스며드는 중이다 (feat.장보기),tistory,나는 여전히 일주일에 한번꼴로 아침일찍 리들에 간다. 오늘은 근처사는 애교많은 치즈...
1,https://donothurt.tistory.com/28,life,건강,"기침에 좋은 음식,은행 잎 열매 효능 부작용",tistory,"안녕하세요,아플때울지마 입니다. 오늘은 요즘들어 제가 잔기침이 많아 기능성 음식에 ..."
2,https://miyah806.tistory.com/entry/93세-엄마의-생신,life,해외생활,엄마의 93세 생신,tistory,2023년7월9일 일요일 Cloudy H:C29도 L:C22도 (H:F 85도 L...


# Naver Blog

## Topic

In [21]:
topic1={
5: "엔터테인먼트·예술",
6: "엔터테인먼트·예술",
7: "엔터테인먼트·예술",
8: "엔터테인먼트·예술",
9: "엔터테인먼트·예술",
10: "엔터테인먼트·예술",
11: "엔터테인먼트·예술",
12: "엔터테인먼트·예술",
13: "엔터테인먼트·예술",
14: "생활·노하우·쇼핑",
15: "생활·노하우·쇼핑",
16: "생활·노하우·쇼핑",
17: "생활·노하우·쇼핑",
18: "생활·노하우·쇼핑",
19: "생활·노하우·쇼핑",
20: "생활·노하우·쇼핑",
21: "생활·노하우·쇼핑",
22: "취미·여가·여행",
23: "취미·여가·여행",
24: "취미·여가·여행",
25: "취미·여가·여행",
26: "취미·여가·여행",
27: "취미·여가·여행",
28: "취미·여가·여행",
29: "취미·여가·여행",
30: "지식·동향",
31: "지식·동향",
32: "지식·동향",
33: "지식·동향",
34: "지식·동향",
35: "지식·동향",
36: "생활·노하우·쇼핑"
}

In [22]:
topic2={
5: "문학·책",
6: "영화",
7: "공연·전시",
8: "미술·디자인",
9: "드라마",
10: "방송",
11: "음악",
12: "스타·연예인",
13: "만화·애니",
14: "일상·생각",
15: "육아·결혼",
16: "반려동물",
17: "좋은글·이미지",
18: "패션·미용",
19: "인테리어·DIY",
20: "요리·레시피",
21: "상품리뷰",
22: "게임",
23: "스포츠",
24: "사진",
25: "자동차",
26: "취미",
27: "국내여행",
28: "세계여행",
29: "맛집",
30: "IT·컴퓨터",
31: "사회·정치",
32: "건강·의학",
33: "비즈니스·경제",
34: "교육·학문",
35: "어학·외국어",
36: "원예·재배"
}

## URL

In [23]:
def makeUrl_NaverBlog(topic,end_pg):
    urls=[]
    for i in range(1,end_pg+1):
        url=f'https://section.blog.naver.com/ThemePost.naver?directoryNo={topic}&activeDirectorySeq=0&currentPage={i}'
        urls.append(url)
    return urls

In [24]:
L_error=[]
def url_NaverBlog(topic=range(5,37),end_pg=10):
    
    """
    NaverBlog에 포함된 'url', 'title', 'big_topic', 'small_topic'을 가져오는 함수
        Args:
            topic (list): 크롤링하고 싶은 주제 | [5,6,7,8, ... ,36]
            end_pg (int): 크롤링할 마지막 페이지
        Returns:
            'url', 'title', 'big_topic', 'small_topic', 'source' 컬럼을 가진 dataframe
    """
    
    df=pd.DataFrame()
    
    for n in topic:
        globals()[f'urls_{n}']=makeUrl_NaverBlog(topic=n,end_pg=end_pg)
    
    for n in topic:
        L_url=[]
        L_title=[]
        L_label=[]

        for url in tqdm(globals()[f'urls_{n}']):
            #driver = webdriver.Chrome()
            driver = webdriver.Chrome(service=service, options=options)
            driver.get(url)
            time.sleep(2)
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            items = soup.select(".item.multi_pic")

            for item in items:
                L_title.append(item.select_one(".desc_inner").text)
                L_url.append(item.select_one(".desc_inner")['href'])
                L_label.append(n)
            
            driver.close()
            time.sleep(3)
            
            if len(L_url)==0:
                L_error.append(n)
                break
                
        print(f'{topic2[n]}({n}): {len(L_url)}')
        df=pd.concat([df,pd.DataFrame({'url':L_url,'label':L_label, 'title':L_title})]).reset_index(drop=True)
        
    df['big_topic']=df['label'].apply(lambda x: topic1[x])
    df['small_topic']=df['label'].apply(lambda x: topic2[x])
    df['source']='Naver_Blog'
    df['title']=df['title'].apply(lambda x: x.replace("\n",""))
    return df

In [25]:
df=url_NaverBlog(topic=range(5,8),end_pg=2)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:15<00:00,  7.62s/it]


문학·책(5): 19


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.99s/it]


영화(6): 20


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.97s/it]

공연·전시(7): 20


In [26]:
df.head(3)

,url,label,title,big_topic,small_topic,source
0,https://blog.naver.com/bahnsoon/223152196964,5,고퀄이던 협재해수욕장 근처 맛집 제주 운치,엔터테인먼트·예술,문학·책,Naver_Blog
1,https://blog.naver.com/thetreasurehouse/223152...,5,어떤 생각들은 나의 세계가 된다(이충녕),엔터테인먼트·예술,문학·책,Naver_Blog
2,https://blog.naver.com/jhji21/223152236910,5,[독서리뷰] 공간디깅: 감도 높은 공간들에 대한 인사이트 모음집 | 공간브랜딩,엔터테인먼트·예술,문학·책,Naver_Blog


## Fix URL

In [27]:
def delete_iframe_NaverBlog(url):
    
    """
    NaverBlog에 포함된 url에 대해 iframe 제거 후 blog.naver.com 붙이는 함수
        Args:
            url : NaverBlog에 포함된 url
        Returns:
            'url', 'url_fix' 컬럼을 가진 dataframe
    """
    
    try:
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "lxml") 
        src_url = "https://blog.naver.com/" + soup.iframe["src"]
        return src_url
    
    except:
        return np.nan

In [28]:
for i in tqdm(range(len(df))):
    df.loc[i,'url_fix']=delete_iframe_NaverBlog(df.loc[i,'url'])

100%|██████████████████████████████████████████████████████████████████████████████████| 59/59 [00:02<00:00, 22.17it/s]


In [30]:
df.head(3)

,url,label,title,big_topic,small_topic,source,url_fix
0,https://blog.naver.com/bahnsoon/223152196964,5,고퀄이던 협재해수욕장 근처 맛집 제주 운치,엔터테인먼트·예술,문학·책,Naver_Blog,https://blog.naver.com//PostView.naver?blogId=...
1,https://blog.naver.com/thetreasurehouse/223152...,5,어떤 생각들은 나의 세계가 된다(이충녕),엔터테인먼트·예술,문학·책,Naver_Blog,https://blog.naver.com//PostView.naver?blogId=...
2,https://blog.naver.com/jhji21/223152236910,5,[독서리뷰] 공간디깅: 감도 높은 공간들에 대한 인사이트 모음집 | 공간브랜딩,엔터테인먼트·예술,문학·책,Naver_Blog,https://blog.naver.com//PostView.naver?blogId=...


## Context

In [31]:
def context_NaverBlog(url):
    
    """
    NaverBlog에 포함된 url에 해당하는 'context'를 가져오는 함수
        Args:
            url_fix : NaverBlog에 포함된 url에 대해 iframe 제거 후 blog.naver.com 붙인 수정된 url
        Returns:
            'url_fix', 'context' 컬럼을 가진 dataframe
    """
    
    try:
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "lxml") 
        text = soup.find("div", attrs={"class":"se-main-container"}).get_text()
        text = text.replace("\n","") #공백 제거
        return text
    except:
        return np.nan

In [32]:
for i in tqdm(range(len(df))):
    df.loc[i,'context']=context_NaverBlog(df.loc[i,'url_fix'])

100%|██████████████████████████████████████████████████████████████████████████████████| 59/59 [00:19<00:00,  2.98it/s]


In [35]:
df.head(3)

,url,label,title,big_topic,small_topic,source,url_fix,context
0,https://blog.naver.com/bahnsoon/223152196964,5,고퀄이던 협재해수욕장 근처 맛집 제주 운치,엔터테인먼트·예술,문학·책,Naver_Blog,https://blog.naver.com//PostView.naver?blogId=...,​얼마 전에 친구들이랑 올 여름 첫 물놀이를 하고 나서 협재해수욕장 근처 맛집에서 ...
1,https://blog.naver.com/thetreasurehouse/223152...,5,어떤 생각들은 나의 세계가 된다(이충녕),엔터테인먼트·예술,문학·책,Naver_Blog,https://blog.naver.com//PostView.naver?blogId=...,어떤 생각들은 나의 세계가 된다 저자이충녕출판위즈덤하우스발매2022.11.11. ​...
2,https://blog.naver.com/jhji21/223152236910,5,[독서리뷰] 공간디깅: 감도 높은 공간들에 대한 인사이트 모음집 | 공간브랜딩,엔터테인먼트·예술,문학·책,Naver_Blog,https://blog.naver.com//PostView.naver?blogId=...,공간디깅 저자theblank_(더블랭크) 편집팀출판스페이스뱅크발매2023.07.03...
